# Remote IBM cloud execution (benchmark)

In [51]:
!pip install qiskit-ibm-runtime

  Using cached qiskit_terra-0.20.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.5 MB)
  Attempting uninstall: qiskit-terra
    Found existing installation: qiskit-terra 0.19.2
    Uninstalling qiskit-terra-0.19.2:
      Successfully uninstalled qiskit-terra-0.19.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
qiskit 0.34.2 requires qiskit-terra==0.19.2, but you have qiskit-terra 0.20.1 which is incompatible.


In [96]:
import os
import time
import json
import numpy as np
from qiskit.providers.jobstatus import JobStatus, JOB_FINAL_STATES
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.accounts.exceptions import AccountAlreadyExistsError

api_token = '740a61f5aadd4bfe5b9d655ac31148c9b08695f3ed4895f612341124c49160386d628465753627f3097c19b1eb6d1c4360a960bbe0243a3deffc316a396ea6a1'

try:
    QiskitRuntimeService.save_account(channel="ibm_quantum", token=api_token)
except AccountAlreadyExistsError as e:
    print('Using existing account')
except:
    print('Coult no save IBM account')
    
qrs = QiskitRuntimeService()
    


Using existing account


In [100]:

def delete_all_jobs(qrs):
    """ Delete all available jobs"""
    
    deleted = 0
    while True:
        jobs = qrs.jobs()
        if len(jobs) == 0:
            break
        
        for job in jobs:
            qrs.delete_job(job.job_id)
        deleted += len(jobs)
    
    print(f'Deleted {deleted} jobs')

    

def run_program(qrs,program_id,program_inputs,options,callback,return_profile_info=False,verbose=True):

    job = qrs.run(
        program_id=program_id,
        options=options,
        inputs=program_inputs,
        callback=result_callback,
    )

    # Printing the job ID in case we need to retrieve it later.
    if verbose:
        print(f"job id: {job.job_id}")

    """
    JobStates enum

        INITIALIZING = "job is being initialized"
        QUEUED = "job is queued"
        VALIDATING = "job is being validated"
        RUNNING = "job is actively running"
        CANCELLED = "job has been cancelled"
        DONE = "job has successfully run"
        ERROR = "job incurred error"
    """

    prev_status = job.status()
    prev_time = time.time()
    start_time = prev_time
    profile_times = {}
    while True:
        s = job.status()
        if s != prev_status:
            dt = time.time() - prev_time
            if verbose:
                print(f'{prev_status} {dt:.2f} sec')
            profile_times[prev_status] = dt
            prev_status = s
            prev_time += dt

        if s in JOB_FINAL_STATES:
            print(f'{s}')
            break
        time.sleep(0.1)
    total_time = time.time() - start_time
    profile_times['TOTAL'] = total_time
    if verbose:
        print(f'Job total time {total_time:.2f}')


    # Get the job result - this is blocking and control may not return immediately.
    result = job.result()
    if return_profile_info:
        return result, profile_times
    else:
        return result

    # Clean job
    qrs.delete_job(job.job_id)

# Specify the program inputs here.
program_inputs = {"iterations": 3}

# Specify the backend name.
options = {"backend_name": "ibmq_qasm_simulator"}


def result_callback(job_id, result):
    #print(f"result: {result}")
    pass

delete_all_jobs(qrs)

results = run_program(qrs,'hello-world',program_inputs,options,result_callback)
print('Results:')
print(results)


Deleted 2 jobs
job id: c9qgponeg6m35kv70pt0
JobStatus.QUEUED 3.36 sec
JobStatus.RUNNING 12.78 sec
JobStatus.DONE
Job total time 16.14
Results:
Hello, World!


In [41]:

# Class to serialize between python / JSON
class JSONSerializer:
    def __init__(self, **kwargs):
        self.params = kwargs

    def to_json(self):
        return json.dumps(self.params)

    @classmethod
    def from_json(cls, json_str):
        return cls(**json.loads(json_str))
    

x = JSONSerializer(param1=3,param2='kkk')
j = x.to_json()
y = JSONSerializer.from_json(j)

In [ ]:
def delete_programs(qrs,text_pattern):
    programs = qrs.programs()
    deleted = 0
    for p in programs:
        pid = p.program_id
        if text_pattern in pid:
            qrs.delete_program(pid)
            deleted += 1

    print(f'Deleted {deleted} programs')

delete_programs(qrs,'basic-program')

program_data = os.path.join(os.getcwd(),'programs/basic_program.py')
program_metadata = os.path.join(os.getcwd(),'programs/basic_program.json')

program_id = qrs.upload_program(data=program_data,metadata=program_metadata)
print(f'Uploaded : {program_id}')

# --- Run program ---
program_inputs = {"iterations": 3}

# Specify the backend name.
options = {"backend_name": "ibmq_qasm_simulator"}


def result_callback(job_id, result):
    #print(f"result: {result}")
    pass

delete_all_jobs(qrs)

RUNS = 5
NUM_QUBITS_RANGE = range(5,60,5)


acc_que = []

for num_qubits in NUM_QUBITS_RANGE:
    acc = []
    for run in range(RUNS):
        program_inputs['num_qubits'] = num_qubits
        program_inputs['depth'] = 10
        results, profile_info = run_program(qrs,program_id,program_inputs,options,result_callback,return_profile_info=True,verbose=False)
        
        if JobStatus.QUEUED in profile_info:
            acc_que.append(profile_info[JobStatus.QUEUED])
            
        acc.append(profile_info[JobStatus.RUNNING])
        
    avg_time, std_time = np.mean(acc), np.std(acc)
    print(f'Num qubits [{num_qubits}] : {avg_time:.3f} +- {std_time:.3f}')





Deleted 0 programs
Uploaded : basic-program-G4YvVQ33vR
Deleted 16 jobs
JobStatus.DONE
JobStatus.DONE
JobStatus.DONE
JobStatus.DONE
JobStatus.DONE
Num qubits [20] : 9.189 +- 3.540
JobStatus.DONE
JobStatus.DONE
JobStatus.DONE
JobStatus.DONE
JobStatus.DONE
Num qubits [25] : 13.393 +- 5.093
JobStatus.DONE
JobStatus.DONE
JobStatus.DONE


In [ ]:
np.mean(acc_que)
np.std(acc_que)

Benchmark (over 5 runs):

- Num qubits [5]  : 9.724  +- 4.766
- Num qubits [10] : 12.550 +- 2.667
- Num qubits [15] : 4.746  +- 2.110
- Num qubits [20] : 9.614  +- 4.486
- Num qubits [25] : 12.908 +- 3.000
- Num qubits [30] : 158.682 +- 8.546